<a href="https://colab.research.google.com/github/jorge-flores-py/PROCESAMIENTO-DE-HABLA-IAPH/blob/main/BOT_V2_JORGE_FLORES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Proyecto: Bots Universitarios

Este proyecto consiste en el desarrollo de tres tipos de **chatbots académicos** capaces de responder preguntas frecuentes de estudiantes universitarios. Se compararon diferentes enfoques para evaluar su rendimiento, precisión y flexibilidad. Los bots se entrenaron con un set de preguntas y respuestas relacionadas a temas institucionales como inscripciones, calificaciones, equivalencias, etc.

---

## 🧠 1. Bot TF-IDF

- **Técnica**: Vectorización de texto usando `TfidfVectorizer` + similitud de coseno.
- **Funcionamiento**: Convierte la pregunta del usuario en un vector y lo compara con los vectores de preguntas conocidas.
- **Ventaja**: Rápido y simple, no requiere modelos complejos.
- **Desventaja**: No capta el significado profundo de las palabras, solo comparaciones por frecuencia.

---

## 🌐 2. Bot con Embeddings

- **Técnica**: `SentenceTransformer` con modelo `all-MiniLM-L6-v2`.
- **Funcionamiento**: Calcula embeddings semánticos de la pregunta y busca la más parecida en el dataset.
- **Ventaja**: Capta relaciones semánticas más profundas entre preguntas similares.
- **Desventaja**: Respuestas fijas, ya que solo selecciona respuestas predefinidas.

---

## 🧩 3. Bot RAG (Retrieval-Augmented Generation)

- **Técnica**: Combinación de búsqueda semántica con generación de texto usando un modelo LLM (`Flan-T5` o `GPT`).
- **Funcionamiento**:
  1. Busca la respuesta más relevante en la base.
  2. Usa esa información como contexto para generar una respuesta personalizada.
- **Ventaja**: Respuestas más naturales y adaptadas a la pregunta del usuario.
- **Desventaja**: Requiere más recursos computacionales y configuración de un modelo generador.

---



In [1]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 41.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [18]:
import re
import numpy as np
import spacy
import pandas as pd
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
nlp = spacy.load("es_core_news_sm")
stopwords_es = set(STOP_WORDS)

## 1) Dataset de preguntas y respuestas para crear un Chatbot para un atencion de una universidad.


In [4]:
faq_data = [
    ("¿Cómo me inscribo a una materia?", "Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'."),
    ("¿Dónde veo mis notas?", "Podés ver tus notas desde el menú 'Historial Académico' en el sistema de autogestión."),
    ("¿Qué pasa si me llevo una materia?", "Deberás recursarla el próximo cuatrimestre."),
    ("¿Puedo cambiarme de turno?", "Sí, durante el período de ajuste podés cambiarte de turno desde el sistema."),
    ("¿Cómo consulto mis faltas?", "Las faltas se consultan desde la sección 'Asistencia' del sistema de alumnos."),
    ("¿Dónde pido el certificado de alumno regular?", "Desde 'Trámites Online' podés solicitarlo y descargarlo."),
    ("¿Cuántas materias puedo cursar por cuatrimestre?", "Podés cursar hasta 5 materias por cuatrimestre."),
    ("¿Qué es el SIU Guaraní?", "Es el sistema de gestión académica donde gestionás tus materias y notas."),
    ("¿Cómo consulto el calendario académico?", "El calendario está disponible en la web oficial del instituto."),
    ("¿Dónde veo mi promedio?", "Tu promedio está disponible en el 'Historial Académico' dentro del sistema."),
    ("¿Qué hago si tengo problemas con el campus virtual?", "Contactá a soporte técnico o enviá un mail a soporte@universidad.edu."),
    ("¿Puedo recursar una materia aprobada?", "No, sólo podés recursar materias desaprobadas."),
    ("¿Qué pasa si abandono una materia?", "Figurarás como ausente y no afectará tu promedio."),
    ("¿Cómo pido una equivalencia?", "Debés completar el formulario de equivalencias y presentarlo con tu plan de estudios."),
    ("¿Cuándo son las inscripciones?", "Las inscripciones se habilitan una semana antes del inicio del cuatrimestre."),
    ("¿Qué es una correlativa?", "Es una materia que debés aprobar antes de cursar otra."),
    ("¿Cómo consulto mi situación académica?", "Desde 'Historial Académico' podés ver tu situación actual."),
    ("¿Dónde encuentro el material de estudio?", "Todo el material está en el campus virtual en cada materia."),
    ("¿Puedo rendir un final libre?", "Sí, pero tenés que anotarte en los turnos establecidos."),
    ("¿Cómo me contacto con un profesor?", "Podés enviarle un mensaje desde el campus o escribirle a su mail institucional.")
]


In [5]:
def limpiar_texto(text):
    text = text.lower()
    text = re.sub(r'[.!¿?¡,"\':“”«»…]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^a-zñáéíóúü\s]', '', text)
    return text

def lematizar_y_filtrar(text):
    doc = nlp(text)
    lemas = [token.lemma_ for token in doc if token.lemma_ not in stopwords_es and token.is_alpha]
    return " ".join(lemas)

def procesar_preguntas(preguntas):
    preguntas_limpias = [limpiar_texto(p) for p in preguntas]
    return [lematizar_y_filtrar(p) for p in preguntas_limpias]

In [6]:
preguntas_raw = [p for p, r in faq_data]         # Lista de todas las preguntas originales
respuestas = [r for p, r in faq_data]            # Lista de todas las respuestas
preguntas_procesadas = procesar_preguntas(preguntas_raw)  # Preprocesamiento completo (limpieza + lematización)


##2) Crear el chatbot utilizando TFIDF y similitud del coseno.

In [7]:
class ChatbotTFIDF:
    def __init__(self, preguntas, respuestas):
        self.vectorizer = TfidfVectorizer()
        self.X = self.vectorizer.fit_transform(preguntas)
        self.respuestas = respuestas

    def responder(self, pregunta_usuario):
        entrada = pregunta_usuario.strip().lower()

        # Detectar saludos o despedidas
        if entrada in ["hola", "buenas", "buenos días", "buenas tardes", "buenas noches"]:
            return "¡Hola! ¿En qué puedo ayudarte?"
        elif entrada in ["gracias", "muchas gracias", "chau", "adiós", "nos vemos", "hasta luego"]:
            return "¡Gracias a vos! ¡Hasta luego!"

        # Proceso normal de respuesta usando TF-IDF
        pregunta_proc = procesar_preguntas([pregunta_usuario])[0]
        pregunta_vec = self.vectorizer.transform([pregunta_proc])
        similitudes = cosine_similarity(pregunta_vec, self.X)
        idx = similitudes.argmax()
        return self.respuestas[idx]


In [121]:
# Ejemplo de uso
chatbot_tfidf = ChatbotTFIDF(preguntas_procesadas, respuestas)
#print(chatbot_tfidf.responder("¿Dónde veo mis calificaciones?"))
print(chatbot_tfidf.responder("como me inscribo"))

Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.


#3) Crear otro chatbot utilizando embeddings. Indique cuál embedding (1 punto) pre-entrenado eligió.

In [9]:
from sentence_transformers import SentenceTransformer
import numpy as np

class ChatbotEmbeddings:
    def __init__(self, data):
        self.modelo = SentenceTransformer('all-MiniLM-L6-v2')
        self.preguntas = [item[0] for item in data]
        self.respuestas = [item[1] for item in data]
        self.embeddings = self.modelo.encode(self.preguntas, convert_to_numpy=True)

    def responder(self, pregunta_usuario):
        entrada = pregunta_usuario.strip().lower()

        # Manejo de saludos o despedidas
        if entrada in ["hola", "buenas", "buenos días", "buenas tardes", "buenas noches"]:
            return "¡Hola! ¿En qué puedo ayudarte?"
        elif entrada in ["gracias", "muchas gracias", "chau", "adiós", "nos vemos", "hasta luego"]:
            return "¡Gracias a vos! ¡Hasta luego!"

        # Búsqueda por embeddings
        emb_usuario = self.modelo.encode([pregunta_usuario], convert_to_numpy=True)[0]
        similitudes = np.dot(self.embeddings, emb_usuario) / (
            np.linalg.norm(self.embeddings, axis=1) * np.linalg.norm(emb_usuario)
        )
        idx = np.argmax(similitudes)
        return self.respuestas[idx]


In [122]:
chatbot_emb = ChatbotEmbeddings(faq_data)

print(chatbot_emb.responder("hola"))
print(chatbot_emb.responder("gracias"))
print(chatbot_emb.responder("¿Que es el SIU Guarani?"))


¡Hola! ¿En qué puedo ayudarte?
¡Gracias a vos! ¡Hasta luego!
Es el sistema de gestión académica donde gestionás tus materias y notas.


##4) Muestra ambos chatbots funcionando (1 punto)

In [125]:
ground_truth = [
    {
        "pregunta": "cómo hago para anotarme en una materia",
        "respuesta_esperada": "Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'."
    },
    {
        "pregunta": "dónde puedo revisar mis calificaciones",
        "respuesta_esperada": "Podés ver tus notas desde el menú 'Historial Académico' en el sistema de autogestión."
    },
    {
        "pregunta": "cuántas materias se pueden hacer por cuatrimestre",
        "respuesta_esperada": "Podés cursar hasta 5 materias por cuatrimestre."
    },
    {
        "pregunta": "qué pasa si abandono una materia",
        "respuesta_esperada": "Figurarás como ausente y no afectará tu promedio."
    },
    {
        "pregunta": "me quiero cambiar de turno, ¿puedo?",
        "respuesta_esperada": "Sí, durante el período de ajuste podés cambiarte de turno desde el sistema."
    },
    {
        "pregunta": "dónde puedo ver mi promedio",
        "respuesta_esperada": "Tu promedio está disponible en el 'Historial Académico' dentro del sistema."
    },
    {
        "pregunta": "cómo se piden las equivalencias",
        "respuesta_esperada": "Debés completar el formulario de equivalencias y presentarlo con tu plan de estudios."
    },
    {
        "pregunta": "me falló el campus virtual, no entra",
        "respuesta_esperada": "Contactá a soporte técnico o enviá un mail a soporte@universidad.edu."
    },
    {
        "pregunta": "cuándo arrancan las inscripciones",
        "respuesta_esperada": "Las inscripciones se habilitan una semana antes del inicio del cuatrimestre."
    },
    {
        "pregunta": "cómo me comunico con un docente",
        "respuesta_esperada": "Podés enviarle un mensaje desde el campus o escribirle a su mail institucional."
    }
]


In [126]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

modelo_similitud = SentenceTransformer("all-MiniLM-L6-v2")

def evaluar_bot(bot, ground_truth, nombre):
    resultados = []

    for item in ground_truth:
        pregunta = item["pregunta"]
        esperado = item["respuesta_esperada"]
        generado = bot.responder(pregunta)

        # Similitud semántica
        emb_esperado = modelo_similitud.encode(esperado, convert_to_tensor=True)
        emb_generado = modelo_similitud.encode(generado, convert_to_tensor=True)
        score = util.cos_sim(emb_esperado, emb_generado).item()

        exacto = generado.strip().lower() == esperado.strip().lower()

        resultados.append({
            "Bot": nombre,
            "Pregunta": pregunta,
            "Esperado": esperado,
            "Generado": generado,
            "Similitud": round(score, 2),
            "Exacto": "✅" if exacto else "❌"
        })

    return pd.DataFrame(resultados)


In [127]:
# Instanciamos ambos bots
bot_tfidf = ChatbotTFIDF(preguntas_procesadas, respuestas)
bot_emb = ChatbotEmbeddings(faq_data)

# Evaluamos
df_tfidf = evaluar_bot(bot_tfidf, ground_truth, "TF-IDF")
df_emb = evaluar_bot(bot_emb, ground_truth, "Embeddings")

# Mostramos todo junto
df_resultado = pd.concat([df_tfidf, df_emb], ignore_index=True)

# Mostrar en tabla
from IPython.display import display
display(df_resultado)


,Bot,Pregunta,Esperado,Generado,Similitud,Exacto
0,TF-IDF,cómo hago para anotarme en una materia,"Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.","Sí, durante el período de ajuste podés cambiarte de turno desde el sistema.",0.50,❌
1,TF-IDF,dónde puedo revisar mis calificaciones,Podés ver tus notas desde el menú 'Historial Académico' en el sistema de autogestión.,"Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.",0.47,❌
2,TF-IDF,cuántas materias se pueden hacer por cuatrimestre,Podés cursar hasta 5 materias por cuatrimestre.,Podés cursar hasta 5 materias por cuatrimestre.,1.00,✅
3,TF-IDF,qué pasa si abandono una materia,Figurarás como ausente y no afectará tu promedio.,Figurarás como ausente y no afectará tu promedio.,1.00,✅
4,TF-IDF,"me quiero cambiar de turno, ¿puedo?","Sí, durante el período de ajuste podés cambiarte de turno desde el sistema.","Sí, durante el período de ajuste podés cambiarte de turno desde el sistema.",1.00,✅
5,TF-IDF,dónde puedo ver mi promedio,Tu promedio está disponible en el 'Historial Académico' dentro del sistema.,Tu promedio está disponible en el 'Historial Académico' dentro del sistema.,1.00,✅
6,TF-IDF,cómo se piden las equivalencias,Debés completar el formulario de equivalencias y presentarlo con tu plan de estudios.,Debés completar el formulario de equivalencias y presentarlo con tu plan de estudios.,1.00,✅
7,TF-IDF,"me falló el campus virtual, no entra",Contactá a soporte técnico o enviá un mail a soporte@universidad.edu.,Contactá a soporte técnico o enviá un mail a soporte@universidad.edu.,1.00,✅
8,TF-IDF,cuándo arrancan las inscripciones,Las inscripciones se habilitan una semana antes del inicio del cuatrimestre.,Las inscripciones se habilitan una semana antes del inicio del cuatrimestre.,1.00,✅
9,TF-IDF,cómo me comunico con un docente,Podés enviarle un mensaje desde el campus o escribirle a su mail institucional.,"Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.",0.42,❌


5) BOT con RAG



In [123]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

class ChatbotRAG:
    def __init__(self, data):
        self.modelo_embedding = SentenceTransformer("all-MiniLM-L6-v2")
        self.preguntas = [item[0] for item in data]
        self.respuestas = [item[1] for item in data]
        self.embeddings = self.modelo_embedding.encode(self.preguntas, convert_to_numpy=True)

        # LLM generador (puede cambiarse por otro más grande o en español)
        self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
        self.llm = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

    def responder(self, pregunta_usuario):
        entrada = pregunta_usuario.strip().lower()

        # Respuestas básicas
        if entrada in ["hola", "buenas", "buenos días", "buenas tardes", "buenas noches"]:
            return "¡Hola! ¿En qué puedo ayudarte?"
        elif entrada in ["gracias", "chau", "nos vemos", "adiós", "hasta luego"]:
            return "¡Gracias a vos! ¡Hasta la próxima!"

        # Paso 1: búsqueda semántica
        emb_usuario = self.modelo_embedding.encode([pregunta_usuario], convert_to_numpy=True)[0]
        similitudes = np.dot(self.embeddings, emb_usuario) / (
            np.linalg.norm(self.embeddings, axis=1) * np.linalg.norm(emb_usuario)
        )
        idx = np.argmax(similitudes)
        contexto = self.respuestas[idx]

        # Paso 2: generación con LLM
        prompt = f"Respondé de forma clara y amable.\nPregunta: {pregunta_usuario}\nContexto: {contexto}\nRespuesta:"

        tokens = self.tokenizer(prompt, return_tensors="pt")
        salida = self.llm.generate(**tokens, max_new_tokens=200)
        respuesta = self.tokenizer.decode(salida[0], skip_special_tokens=True)

        return respuesta


In [131]:
bot_rag = ChatbotRAG(faq_data)
print(bot_rag.responder("COMO me anoto"))


Debés ingresar la sistema de autogestión, ir la sección 'Materias' y seleccionar 'Inscribirse'. Answered


In [129]:
modelo_similitud = SentenceTransformer("all-MiniLM-L6-v2")

def evaluar_bot(bot, ground_truth, nombre):
    resultados = []

    for item in ground_truth:
        pregunta = item["pregunta"]
        esperado = item["respuesta_esperada"]
        generado = bot.responder(pregunta)

        # Similitud semántica
        emb_esperado = modelo_similitud.encode(esperado, convert_to_tensor=True)
        emb_generado = modelo_similitud.encode(generado, convert_to_tensor=True)
        score = util.cos_sim(emb_esperado, emb_generado).item()

        exacto = generado.strip().lower() == esperado.strip().lower()

        resultados.append({
            "Bot": nombre,
            "Pregunta": pregunta,
            "Esperado": esperado,
            "Generado": generado,
            "Similitud": round(score, 2),
            "Exacto": "✅" if exacto else "❌"
        })

    return pd.DataFrame(resultados)


In [29]:
df_tfidf = evaluar_bot(bot_tfidf, ground_truth, "TF-IDF")
df_emb = evaluar_bot(bot_emb, ground_truth, "Embeddings")
df_rag = evaluar_bot(bot_rag, ground_truth, "RAG + LLM")

df_final = pd.concat([df_tfidf, df_emb, df_rag], ignore_index=True)

# Mostrar tabla comparativa
import pandas as pd

# Configuración global de pandas para mostrar todo el contenido
pd.set_option('display.max_colwidth', None)      # No limitar el largo de cada celda
pd.set_option('display.max_columns', None)       # Mostrar todas las columnas
pd.set_option('display.expand_frame_repr', False)  # Mostrar todas las columnas en una sola línea

# Ahora mostrás la tabla normalmente
from IPython.display import display
display(df_final)


,Bot,Pregunta,Esperado,Generado,Similitud,Exacto
0,TF-IDF,cómo hago para anotarme en una materia,"Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.","Sí, durante el período de ajuste podés cambiarte de turno desde el sistema.",0.50,❌
1,TF-IDF,dónde puedo revisar mis calificaciones,Podés ver tus notas desde el menú 'Historial Académico' en el sistema de autogestión.,"Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.",0.47,❌
2,TF-IDF,cuántas materias se pueden hacer por cuatrimestre,Podés cursar hasta 5 materias por cuatrimestre.,Podés cursar hasta 5 materias por cuatrimestre.,1.00,✅
3,TF-IDF,qué pasa si abandono una materia,Figurarás como ausente y no afectará tu promedio.,Figurarás como ausente y no afectará tu promedio.,1.00,✅
4,TF-IDF,"me quiero cambiar de turno, ¿puedo?","Sí, durante el período de ajuste podés cambiarte de turno desde el sistema.","Sí, durante el período de ajuste podés cambiarte de turno desde el sistema.",1.00,✅
5,TF-IDF,dónde puedo ver mi promedio,Tu promedio está disponible en el 'Historial Académico' dentro del sistema.,Tu promedio está disponible en el 'Historial Académico' dentro del sistema.,1.00,✅
6,TF-IDF,cómo se piden las equivalencias,Debés completar el formulario de equivalencias y presentarlo con tu plan de estudios.,Debés completar el formulario de equivalencias y presentarlo con tu plan de estudios.,1.00,✅
7,TF-IDF,"me falló el campus virtual, no entra",Contactá a soporte técnico o enviá un mail a soporte@universidad.edu.,Contactá a soporte técnico o enviá un mail a soporte@universidad.edu.,1.00,✅
8,TF-IDF,cuándo arrancan las inscripciones,Las inscripciones se habilitan una semana antes del inicio del cuatrimestre.,Las inscripciones se habilitan una semana antes del inicio del cuatrimestre.,1.00,✅
9,TF-IDF,cómo me comunico con un docente,Podés enviarle un mensaje desde el campus o escribirle a su mail institucional.,"Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.",0.42,❌


In [35]:
!pip install openai


## BOT RAG USANDO OPENAI

In [86]:
from openai import OpenAI
from google.colab import userdata
from sentence_transformers import SentenceTransformer
import numpy as np


# 🔐 Configuración segura de la API
api_key = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)
#client = OpenAI(api_key="OPENAI_API_KEY")  # Reemplazá con tu clave privada real

class ChatbotRAGOpenAI:
    def __init__(self, data):
        self.modelo_embedding = SentenceTransformer("all-MiniLM-L6-v2")
        self.preguntas = [item[0] for item in data]
        self.respuestas = [item[1] for item in data]
        self.embeddings = self.modelo_embedding.encode(self.preguntas, convert_to_numpy=True)

    def responder(self, pregunta_usuario):
        entrada = pregunta_usuario.strip().lower()

        # Respuestas básicas
        if entrada in ["hola", "buenas", "buenos días", "buenas tardes", "buenas noches"]:
            return "¡Hola! ¿En qué puedo ayudarte?"
        elif entrada in ["gracias", "chau", "nos vemos", "adiós", "hasta luego"]:
            return "¡Gracias a vos! ¡Hasta la próxima!"

        # P Búsqueda semántica
        emb_usuario = self.modelo_embedding.encode([pregunta_usuario], convert_to_numpy=True)[0]
        similitudes = np.dot(self.embeddings, emb_usuario) / (
            np.linalg.norm(self.embeddings, axis=1) * np.linalg.norm(emb_usuario)
        )
        idx = np.argmax(similitudes)
        contexto = self.respuestas[idx]

        # Paso 2: Generación con OpenAI (nuevo cliente)
        prompt = f"""Respondé en español de forma clara, amable y breve. Sos un asistente para estudiantes universitarios y solo podés responder preguntas relacionadas con la institución.

                    Usá exclusivamente la información proporcionada a continuación.
                    Si la pregunta del estudiante no tiene relación con este contenido, respondé: "Lo siento, solo puedo responder consultas relacionadas con la institución."

                    Información disponible: {contexto}
                    Pregunta del estudiante: {pregunta_usuario}
                    Respuesta:"""

        try:
            respuesta = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "user", "content": prompt}
                ],
                temperature=0.4,
                max_tokens=100
            )
            return respuesta.choices[0].message.content.strip()
        except Exception as e:
            return f"Error al generar respuesta: {e}"


In [84]:
bot_openai = ChatbotRAGOpenAI(faq_data)


In [132]:
pregunta = "cómo ido equivalenencias"
respuesta = bot_openai.responder(pregunta)
print("Pregunta:", pregunta)
print("Respuesta:", respuesta)


Pregunta: cómo ido equivalenencias
Respuesta: Debés completar el formulario de equivalencias y presentarlo con tu plan de estudios.


In [66]:
# Instanciás los bots
bot_tfidf = ChatbotTFIDF(preguntas_procesadas, respuestas)
bot_emb = ChatbotEmbeddings(faq_data)
bot_openai = ChatbotRAGOpenAI(faq_data)

# Evaluás
df_tfidf = evaluar_bot(bot_tfidf, ground_truth, "TF-IDF")
df_emb = evaluar_bot(bot_emb, ground_truth, "Embeddings")
df_rag = evaluar_bot(bot_openai, ground_truth, "RAG-OpenAI")

# Concatenás resultados
df_final = pd.concat([df_tfidf, df_emb, df_rag], ignore_index=True)

# Mostrás en Colab
from IPython.display import display
display(df_final)


,Bot,Pregunta,Esperado,Generado,Similitud,Exacto
0,TF-IDF,cómo hago para anotarme en una materia,"Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.","Sí, durante el período de ajuste podés cambiarte de turno desde el sistema.",0.50,❌
1,TF-IDF,dónde puedo revisar mis calificaciones,Podés ver tus notas desde el menú 'Historial Académico' en el sistema de autogestión.,"Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.",0.47,❌
2,TF-IDF,cuántas materias se pueden hacer por cuatrimestre,Podés cursar hasta 5 materias por cuatrimestre.,Podés cursar hasta 5 materias por cuatrimestre.,1.00,✅
3,TF-IDF,qué pasa si abandono una materia,Figurarás como ausente y no afectará tu promedio.,Figurarás como ausente y no afectará tu promedio.,1.00,✅
4,TF-IDF,"me quiero cambiar de turno, ¿puedo?","Sí, durante el período de ajuste podés cambiarte de turno desde el sistema.","Sí, durante el período de ajuste podés cambiarte de turno desde el sistema.",1.00,✅
5,TF-IDF,dónde puedo ver mi promedio,Tu promedio está disponible en el 'Historial Académico' dentro del sistema.,Tu promedio está disponible en el 'Historial Académico' dentro del sistema.,1.00,✅
6,TF-IDF,cómo se piden las equivalencias,Debés completar el formulario de equivalencias y presentarlo con tu plan de estudios.,Debés completar el formulario de equivalencias y presentarlo con tu plan de estudios.,1.00,✅
7,TF-IDF,"me falló el campus virtual, no entra",Contactá a soporte técnico o enviá un mail a soporte@universidad.edu.,Contactá a soporte técnico o enviá un mail a soporte@universidad.edu.,1.00,✅
8,TF-IDF,cuándo arrancan las inscripciones,Las inscripciones se habilitan una semana antes del inicio del cuatrimestre.,Las inscripciones se habilitan una semana antes del inicio del cuatrimestre.,1.00,✅
9,TF-IDF,cómo me comunico con un docente,Podés enviarle un mensaje desde el campus o escribirle a su mail institucional.,"Debés ingresar al sistema de autogestión, ir a la sección 'Materias' y seleccionar 'Inscribirse'.",0.42,❌


In [119]:
from IPython.display import display, HTML
import ipywidgets as widgets

def lanzar_interfaz_chatbot(bot):
    estilo_css = """
    <style>
        .chatbox-container {
            background-color: #000000;
            padding: 15px;
            border-radius: 10px;
            border: 2px solid #00ff00;
            font-family: Arial, sans-serif;
            color: white;
            width: 600px;
            margin-left: 0;
        }
        .titulo {
            font-size: 24px;
            font-weight: bold;
            margin-bottom: 10px;
            color: #00ff00;
        }
        .mensajes-box {
            height: 200px;
            overflow-y: auto;
            border: 1px solid #00ff00;
            padding: 10px;
            border-radius: 5px;
            background-color: #111111;
            font-size: 15px;
            margin-bottom: 10px;
            width: 560px;
        }
        .input-area {
            display: flex;
            justify-content: flex-start;
            align-items: center;
            gap: 5px;
            width: 560px;
        }
        .footer {
            color: #888888;
            font-size: 12px;
            margin-top: 5px;
            margin-left: 0;
            text-align: left;
            width: 560px;
        }
    </style>
    """

    display(HTML(estilo_css))

    mensajes_html = widgets.HTML(value="<i>Escribí una pregunta para comenzar...</i>")
    entrada_usuario = widgets.Text(
        placeholder="Escribí tu pregunta...",
        layout=widgets.Layout(width="450px", height="30px")
    )
    boton_enviar = widgets.Button(
        description="Enviar",
        button_style="success",
        layout=widgets.Layout(width="80px", height="30px")
    )

    historial = []

    def actualizar_chatbox(_):
        pregunta = entrada_usuario.value.strip()
        if pregunta == "":
            return

        respuesta = bot.responder(pregunta)
        historial.append(
            f"<b style='color:#00ffff;'>👤 Vos:</b> {pregunta}<br>"
            f"<b style='color:#00ff00;'>🤖 Botsito:</b> {respuesta}<br><br>"
        )
        mensajes_html.value = "<div class='mensajes-box'>" + "".join(historial) + "</div>"
        entrada_usuario.value = ""

    boton_enviar.on_click(actualizar_chatbox)

    # Interfaz principal
    chat_ui = widgets.VBox([
        widgets.HTML("""
        <div class='chatbox-container'>
            <div class='titulo'>🤖 Chatbot de Asistencia Académica</div>
        """),
        mensajes_html,
        widgets.HBox([entrada_usuario, boton_enviar], layout=widgets.Layout(width="560px")),
        widgets.HTML("</div>"),
        widgets.HTML("<div class='footer'>Desarrollado con tecnología de <b>OpenAI</b></div>")
    ])

    display(chat_ui)


In [133]:
# Usá uno de tus bots
lanzar_interfaz_chatbot(bot_openai)  # o bot_tfidf, o bot_openai


## Conclusiones

El bot que usa la técnica RAG fue el que mejor respondió, porque permite ajustar cómo queremos que conteste a través del prompt. Esto hace que las respuestas sean más naturales y precisas.

Además, notamos que el modelo de OpenAI (GPT) funcionó mejor que el de Google (Flan-T5), ya que dio respuestas más claras y coherentes.



## 📚 Referencias

- 🔗 **Video tutorial 1:**  
  *Cómo conectar con la API de ChatGPT (OpenAI)*  
  [https://www.youtube.com/watch?v=HayksI4d1Q8](https://www.youtube.com/watch?v=HayksI4d1Q8)

- 🔐 **Video tutorial 2:**  
  *Cómo agregar tokens como secretos en Google Colab*  
  [https://www.youtube.com/watch?v=LPa51KxqUAw](https://www.youtube.com/watch?v=LPa51KxqUAw)

- 🧠 **Herramienta utilizada:**  
  Se utilizó **ChatGPT** (modelo GPT-4 de OpenAI) como asistente de desarrollo y redacción para generar código, explicaciones y soporte técnico durante el proyecto.

- 📝 **Apuntes de clase:**  
  Se consultaron materiales teóricos y prácticos provistos en clase sobre NLP (Procesamiento del Lenguaje Natural), uso de embeddings y sistemas RAG (Retrieval-Augmented Generation).



In [138]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [140]:
notebook_path = "/content/drive/MyDrive/Colab Notebooks/BOT_V2_JORGE_FLORES.ipynb"

